In [1]:
import numpy as np
from scipy.signal import stft, get_window
import matplotlib.pyplot as plt
import os
import cv2
import math

In [2]:
def compute_spectrogram(cir_data, window_size, window_step, doppler_bins, hanning_window, T, N):
	spectrograms = None

	for n in range(N):

		s_n = cir_data[:, n]

		f, t, Zxx = stft(
        s_n,
        fs=1.0,
        window=hanning_window,
        nperseg=window_size,
        noverlap=window_size - window_step,
        nfft=doppler_bins,
        boundary=None,
        padded=False,
        return_onesided=False
    	)

		Zxx_shifted = np.fft.fftshift(Zxx, axes=0)
		magnitude_spectrum = np.abs(Zxx_shifted)

		if spectrograms is None:
			num_time_frames = magnitude_spectrum.shape[1]
			spectrograms = np.zeros((doppler_bins, num_time_frames, N), dtype=np.float32)

		spectrograms[:, :, n] = magnitude_spectrum

	spectrograms_dB = 20 * np.log10(spectrograms)

	spectrograms_normalized = np.zeros_like(spectrograms_dB)

	for t_idx in range(num_time_frames):
		for n in range(N):
			spectrum = spectrograms_dB[:, t_idx, n]

			min_val = np.min(spectrum)
			max_val = np.max(spectrum)
			normalized_spectrum = (spectrum - min_val) / (max_val - min_val)
	        
			spectrograms_normalized[:, t_idx, n] = normalized_spectrum
	
	return spectrograms_normalized



def agregate_spectrogram(spectrumm, threshold_k=None):
	# Sum the spectrogram over time and frequency to get the energy per range bin
	energy_per_range_bin = np.sum(spectrumm, axis=(0, 1))

	# Identify range bins with energy above a certain threshold
	threshold = np.mean(energy_per_range_bin) + threshold_k * np.std(energy_per_range_bin)
	target_range_bins = np.where(energy_per_range_bin > threshold)[0]

	print(f"Target Range Bins: {target_range_bins}")

	# Aggregate spectrograms over target range bins
	aggregated_spectrogram = np.mean(spectrumm[:, :, target_range_bins], axis=2)

	min_val = np.min(aggregated_spectrogram)
	max_val = np.max(aggregated_spectrogram)
	normalized_aggregated = (aggregated_spectrogram - min_val) / (max_val - min_val)
    
	return normalized_aggregated



In [3]:

signatures_path = os.path.join("data", "b_np")
if not os.path.exists(signatures_path):
    raise FileNotFoundError(f"The directory {signatures_path} does not exist.")

output_dir = os.path.join("data", "b_sig_fig")
os.makedirs(output_dir, exist_ok=True)

signatures_output_dir = os.path.join("data", "b_sig")
os.makedirs(signatures_output_dir, exist_ok=True)

for file in os.listdir(signatures_path):
    if not file.endswith(".npy"):
        continue

    cir_data = np.load(os.path.join(signatures_path, file))

    T, N = cir_data.shape

    window_size = 64
    window_step = 32
    doppler_bins = 64
    hanning_window = get_window('hann', window_size)  # Hanning window

    normalized_spectrogram = compute_spectrogram(cir_data, window_size, window_step, doppler_bins, hanning_window, T, N)

    aggregated_spectrogram = agregate_spectrogram(normalized_spectrogram, threshold_k=0.5)

    # Save the aggregated spectrogram as .npy file
    sig_output_filename = os.path.join(signatures_output_dir, file)
    np.save(sig_output_filename, aggregated_spectrogram)

    # Resize the spectrogram to 256x256
    resized_image = cv2.resize(aggregated_spectrogram, (256, 256), interpolation=cv2.INTER_LINEAR)
    
    # Save the image
    output_filename = file.replace('.npy', '.png')
    output_path = os.path.join(output_dir, output_filename)
    plt.imsave(output_path, resized_image, cmap='gray', format='png')

Target Range Bins: [  7  14  22  23  24  25  26  27  29  40  41  42  43  44  46  47  48  49
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  71  72  73
  74  75  76  77  81  82  83  84  85  86  87  88  89  90  91  94  98  99
 100 101 102 103 104 105 106 112 113 114 115 116 117 118 119 120 121 122
 123 128 129 130 131 136 137 138 147 151 164 167 183 215 219 229 230 231
 232 235 236 238 239 243 244 245 246 247 248 250 260 261 262 263 264 283
 295 312 314 315 327 343 368 372 373 374 375 392 396 399 400 401 404 412
 413 414 415 416 419 423 428 429 430 431 432 443 455 456 458 459 471 475
 482 483 484 485 487 488 490 491 492 493 494 497 507 511]
Target Range Bins: [  7  14  15  17  22  23  24  25  26  27  30  40  41  42  43  44  47  55
  56  57  58  59  60  61  62  63  64  67  68  69  70  71  72  73  74  75
  76  81  82  83  84  85  86  87  92  93  94  98  99 100 101 103 104 105
 106 108 109 110 111 112 113 114 115 116 117 118 119 120 123 126 128 129
 130 134 135 136 137 138 139